# Assignment09
# Multi-label classification using neural networks
* 2019220177 Seungwook

## Initial Setting

### Mount Google Drive

In [16]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
%cd /content/drive/My\ Drive/MIMP

/content/drive/My Drive/MIMP


### git Setting

In [18]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   Assignment09/Assignment09.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [0]:
!git config --global user.email "sooonchang@gmail.com"
!git config --global user.name "SooonChang"

In [0]:
!git add .
!git commit -m "submit"
!git push origin master

[master 19472cd] submit
 3 files changed, 10002 insertions(+), 1 deletion(-)
 rewrite Assignment08/Assignment08.ipynb (96%)
 create mode 100644 Assignment09/Assignment09.ipynb
 create mode 100644 Assignment09/mnist.csv
Counting objects: 6, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 2.30 KiB | 261.00 KiB/s, done.
Total 6 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/SooonChang/MIMP
   5f207f0..19472cd  master -> master


## 0. Optimization

### Import Libraries

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict

### Load Data


In [0]:
file_data = "Assignment09/mnist.csv"
handle_file = open(file_data, "r")
data = handle_file.readlines()
handle_file.close()

size_row = 28
size_col = 28

num_image = len(data)
count = 0

list_image = np.empty((size_row* size_col, num_image), dtype=float)
list_label = np.empty(num_image, dtype = int)

for line in data:
  line_data = line.split(',')
  label = line_data[0]
  im_vector = np.asfarray(line_data[1:])

  list_label[count] = label
  list_image[:,count] = im_vector
  count +=1


### define module

In [0]:
def sigmoid(x):
  return 1/ (1 + np.exp(-x))

class Sigmoid():
  
  def __init__(self):
    self.out = None
  
  def forward(self, x):
    out = sigmoid(x)
    self.out = out
    return out
  
  def backward(self, dout): # s' = (1-s)*s
    dx = dout*(1.0 - self.out) * self.out

    return dx

class Layer():
  def __init__(self,W,b):
    self.W = W
    self.b = b

    self.x = None
    self.x_shape = None
    # differntial
    self.dW = None
    self.db = None

  def forward(self, x):
    self.x_shape = x.shape
    x = x.reshape(x.shape[0],-1)
    self.x = x

    out = np.dot(self.x,self.W) + self.b

    return out
  
  def backword(self,dout):
    dx = np.dot(dout,self.W.T)
    self.dW = np.dot(self.x.T,dout)
    self.db = np.sum(dout, axis=0)

    dx = dx.reshape(*self.x_shape)
    return dx

class object_func():
  def __init__(self):
    self.loss = None
    self.h = None
    self.t = None

  def forward(self, y, t):
    self.t = t
    self.h = sigmoid(y)
    m = t.shape[0]
    self.loss = np.sum(-t*np.log(y +1e-7) -(1-t)*np.log(1-y +1e-7)) / m

    return self.loss
  
  def backward(self, dout=1):
    m = self.t.shape[0]
    dx = (self.h - self.t)/m
    
    return dx


In [0]:
class Network():

  def __init__(self, std = 0.01):
    # init weights
    self.params = {}
    self.params['W1'] = std*np.random.randn(784,196)
    self.params['b1'] = np.zeros(196)
    self.params['W2'] = std*np.random.randn(196,49)
    self.params['b2'] = np.zeros(49)
    self.params['W3'] = std*np.random.randn(49,10)
    self.params['b3'] = np.zeros(10)

    # create layers
    self.layers = OrderedDict()
    self.layers['Layer1'] = Layer(self.params['W1'], self.params['b1'])
    self.layers['Sigmoid1'] = Sigmoid()
    self.layers['Layer2'] = Layer(self.params['W2'], self.params['b2'])
    self.layers['Sigmod2'] = Sigmoid()
    self.layers['Layer3'] = Layer(self.params['W3'], self.params['b3'])

    self.lastLayer = object_func()
  
  def predict(self, x):
    for layer in self.layers.values():
      x = layer.forward(x)
    
    return x
  
  def loss(self, x, t):
    y = self.predict(x)
    return self.lastLayer.forward(y,t)
  
  def accuracy(self, x, t):
    y = self.predict(x)
    y = np.argmax(y, axis=1)
    t = np.argmax(t, axis=1)

    accuracy = np.sum(y==t)/ float(x.shape[0])
    return accuracy

  def gradient(self, x, t):
    # forward
    self.loss(x,t)

    # backward
    dout =1
    dout = self.lastLayer.backward(dout)

    layers = list(self.layers.values())
    layres.reverse()
    for layer in layers:
      dout = layer.backward(dout)
    
    # save gradients
    grads = {}
    grads['W1'],grads['b1'] = self.layers['Layer1'].dW, self.layers['Layer1'].db
    grads['W2'],grads['b2'] = self.layers['Layer2'].dw,self.layers['Layer2'].db
    grads['W3'],grads['b3'] = self.layers['Layer3'].dw,self.layers['Layer3'].db

    return grads